<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>


---

# **Exercícios**

Este *notebook* deve servir como um guia para **você continuar** a construção da sua própria análise exploratória de dados interativa. Fique a vontate para copiar os códigos da aula mas busque explorar os dados ao máximo. Por fim, publique seu *notebook* no [Kaggle](https://www.kaggle.com/) e seu *dashboard* [Google Data Studio](https://datastudio.google.com/).

---

# **COVID Dashboard**

## 1\. Contexto

A COVID-19 é uma infecção respiratória aguda causada pelo coronavírus SARS-CoV-2, potencialmente grave, de elevada transmissibilidade e de distribuição global. Fonte: Governo brasileiro.

A disponibilidade de dados sobre a evolução da pandemia no tempo em uma determinada região geográfica é fundamental para o seu combate! Este projeto busca construir um dashboard de dados para exploração e visualização interativa de dados sobre o avanço de casos e da vacinação do Brasil.

Os dados sobre casos da COVID-19 são compilados pelo centro de ciência de sistemas e engenharia da universidade americana John Hopkins (https://www.jhu.edu/). Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de regiões de países (estados, condados, etc.)

**date**: data de referência;

**state**: estado;

**country**: país;

**population**: população estimada;

**confirmed**: número acumulado de infectados;

**confirmed_1d**: número diário de infectados;

**confirmed_moving_avg_7d**: média móvel de 7 dias do número diário de infectados;

**confirmed_moving_avg_7d_rate_14d**: média móvel de 7 dias
dividido pela média móvel de 7 dias de 14 dias atrás;

**deaths**: número acumulado de mortos;

**deaths_1d**: número diário de mortos;

**deaths_moving_avg_7d**: média móvel de 7 dias do número diário de mortos;

**deaths_moving_avg_7d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;

**month**: mês de referência;

**year**: ano de referência.

os dados sobre a vacinação foram retirados do  https://opendatasus.saude.gov.br/, do qual disponibiliza dados sobre a SUS (Sistema Único de Saúde)


**state**: Cidade

**date**: Data a vacinação

**primeira_dose**: quantidade de aplicações da primeira dose no dia

**segunda_dose**: quantidade de aplicações da segunda dose no dia

**terceira_dose**: quantidade de aplicações da terceia dose no dia

**population**: população estimada

**primeira_dose_acumulado**: quantidade acumuadalo da primeira dose

**segunda_dose_acumulado**: quantidade acumuadalo da segunda dose

**terceira_dose_acumulado**: quantidade acumuadalo da terceira dose

**primeira_dose_acumulado%**: percentual de vacinação com a primeira dose

**segunda_dose_acumulado%**: percentual de vacinação com a segunda dose

**terceira_dose_acumulado%**: percentual de vacinação com a terceira dose

**primeira_dose_moving_avg_7d**:  média móvel de 7 dias do número diário de vacinados com a primeira dose

**primeira_dose_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;

**segunda_dose_moving_avg_7d**:	média móvel de 7 dias do número diário de vacinados com a segunda dose

**segunda_dose_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;

**terceira_dose_moving_avg_7d**	média móvel de 7 dias do número diário de vacinados com a terceira dose

**terceira_dose_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;

**month**: mês de referência;

**year**: ano de referência.

## 2\. Pacotes e bibliotecas

Nesta sessão vamos utilizar os seguintes pacotes Python para processar os dados bruto em um formato adequado para um painel para exploração interativa de dados.

In [ ]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

## 3\. Extração

O dado está compilado em um arquivo por dia, exemplo para 2021/12/01.



Portanto, precisaremos iterar dentro de um intervalo de tempo definido para extraí-lo.



In [ ]:
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

De maneira iterativa, vamos selecionar as colunas de interesse e as linhas referentes ao Brasil.



In [ ]:
start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)


cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = pd.concat([cases, case], ignore_index=True)

In [ ]:
cases.query('Province_State == "Sao Paulo"').head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
24,Sao Paulo,Brazil,1466191,46775,3192.990778,2021-01-01
51,Sao Paulo,Brazil,1467953,46808,3196.827966,2021-01-02
78,Sao Paulo,Brazil,1471422,46845,3204.382565,2021-01-03
105,Sao Paulo,Brazil,1473670,46888,3209.278136,2021-01-04
132,Sao Paulo,Brazil,1486551,47222,3237.329676,2021-01-05


In [ ]:
cases.head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01


In [ ]:
cases.shape

(9828, 6)

In [ ]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province_State  9828 non-null   object        
 1   Country_Region  9828 non-null   object        
 2   Confirmed       9828 non-null   int64         
 3   Deaths          9828 non-null   int64         
 4   Incident_Rate   9828 non-null   float64       
 5   Date            9828 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 460.8+ KB


In [ ]:
vaccines = pd.read_csv('./vaccines_2021.csv',parse_dates=['date'], sep=';')

In [ ]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9599 entries, 0 to 9598
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   state          9599 non-null   object        
 1   date           9599 non-null   datetime64[ns]
 2   primeira_dose  9599 non-null   int64         
 3   segunda_dose   9599 non-null   int64         
 4   terceira_dose  9599 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 375.1+ KB


## 4\. Transformação

Ajustando os dados para terem uma boa consistência e atender as nescessídades


In [ ]:
# Ajustando os nomes das culunas


cases = cases.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

In [ ]:
# Padronizando os estados

states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Maranhao': 'Maranhão',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

In [ ]:
# criando novas colunas com o mês referência.


cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))

In [ ]:
# criando a coluna população


cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

In [ ]:
# Criando as colunas de médias moveis

cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.75:
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status


for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = pd.concat([cases_, cases_per_state], ignore_index=True)

cases = cases_
cases_ = None

Formato e organizo as colunas do DataSet

In [ ]:
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

In [ ]:
cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]
cases.head(25)

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,<NA>,NaN,NaN,796,<NA>,<NA>,NaN,NaN,2021-01,2021
1,2021-01-02,Brazil,Acre,881935,41941,252,<NA>,NaN,NaN,798,2,<NA>,NaN,NaN,2021-01,2021
2,2021-01-03,Brazil,Acre,881935,42046,105,<NA>,NaN,NaN,802,4,<NA>,NaN,NaN,2021-01,2021
3,2021-01-04,Brazil,Acre,881935,42117,71,<NA>,NaN,NaN,806,4,<NA>,NaN,NaN,2021-01,2021
4,2021-01-05,Brazil,Acre,881935,42170,53,<NA>,NaN,NaN,808,2,<NA>,NaN,NaN,2021-01,2021
5,2021-01-06,Brazil,Acre,881935,42378,208,<NA>,NaN,NaN,814,6,<NA>,NaN,NaN,2021-01,2021
6,2021-01-07,Brazil,Acre,881935,42478,100,<NA>,NaN,NaN,821,7,<NA>,NaN,NaN,2021-01,2021
7,2021-01-08,Brazil,Acre,881935,42814,336,161,NaN,NaN,823,2,4,NaN,NaN,2021-01,2021
8,2021-01-09,Brazil,Acre,881935,42908,94,139,NaN,NaN,823,0,4,NaN,NaN,2021-01,2021
9,2021-01-10,Brazil,Acre,881935,43127,219,155,NaN,NaN,825,2,4,NaN,NaN,2021-01,2021


Adiciono ao data vaccines a coluna de população

In [ ]:
pop_state = cases[['date','state','population']]

In [ ]:
vaccines = pd.merge(vaccines, pop_state, how='left', on=['date','state'])

Removo os dia 31 de dezembro, pois não tem as informações completas

In [ ]:
states_null = vaccines[vaccines['population'].isnull()]

states_null

,state,date,primeira_dose,segunda_dose,terceira_dose,population
348,Acre,2021-12-31,28,253,0,<NA>
697,Alagoas,2021-12-31,11,307,0,<NA>
1046,Amapá,2021-12-31,12,56,0,<NA>
1400,Amazonas,2021-12-31,68,359,0,<NA>
1763,Bahia,2021-12-31,94,1040,0,<NA>
2127,Ceará,2021-12-31,30,233,0,<NA>
2479,Distrito Federal,2021-12-31,102,2708,0,<NA>
2844,Espírito Santo,2021-12-31,73,1569,0,<NA>
3197,Goiás,2021-12-31,29,814,0,<NA>
3551,Maranhão,2021-12-31,54,352,0,<NA>


In [ ]:
vaccines.dropna(inplace=True)

In [ ]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9572 entries, 0 to 9597
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   state          9572 non-null   object        
 1   date           9572 non-null   datetime64[ns]
 2   primeira_dose  9572 non-null   int64         
 3   segunda_dose   9572 non-null   int64         
 4   terceira_dose  9572 non-null   int64         
 5   population     9572 non-null   Int64         
dtypes: Int64(1), datetime64[ns](1), int64(3), object(1)
memory usage: 532.8+ KB


Crio as colunas de acompanhamento da vacinação.

In [ ]:
doses = ['primeira_dose',	'segunda_dose',	'terceira_dose'	]

for dose in doses:
  for state in vaccines['state'].unique():
      for date in vaccines['date'].unique():

          total_doses =round(vaccines[(vaccines['date'] <= date) & (vaccines['state'] == state)][dose].sum())
          vaccines.loc[(vaccines['date'] == date) & (vaccines['state'] == state), f'{dose}_acumulado'] = total_doses



In [ ]:
doses_acum = ['primeira_dose_acumulado',	'segunda_dose_acumulado',	'terceira_dose_acumulado'	]

for dose in doses_acum:
  vaccines[f'{dose}%'] = round(vaccines[dose] / vaccines['population']*100,3)

In [ ]:
vaccines.head(10)

,state,date,primeira_dose,segunda_dose,terceira_dose,population,primeira_dose_acumulado,segunda_dose_acumulado,terceira_dose_acumulado,primeira_dose_acumulado%,segunda_dose_acumulado%,terceira_dose_acumulado%
0,Acre,2021-01-17,5,0,0,881935,5.0,0.0,0.0,0.001,0.0,0.0
1,Acre,2021-01-18,5,0,0,881935,10.0,0.0,0.0,0.001,0.0,0.0
2,Acre,2021-01-19,26,0,0,881935,36.0,0.0,0.0,0.004,0.0,0.0
3,Acre,2021-01-20,947,2,0,881935,983.0,2.0,0.0,0.111,0.0,0.0
4,Acre,2021-01-21,1326,2,0,881935,2309.0,4.0,0.0,0.262,0.0,0.0
5,Acre,2021-01-22,951,4,0,881935,3260.0,8.0,0.0,0.37,0.001,0.0
6,Acre,2021-01-23,154,2,0,881935,3414.0,10.0,0.0,0.387,0.001,0.0
7,Acre,2021-01-24,84,2,0,881935,3498.0,12.0,0.0,0.397,0.001,0.0
8,Acre,2021-01-25,752,0,0,881935,4250.0,12.0,0.0,0.482,0.001,0.0
9,Acre,2021-01-26,981,1,0,881935,5231.0,13.0,0.0,0.593,0.001,0.0


In [ ]:
# as médias moveis de vacinação.

vaccines_ = None
vaccines_is_empty = True


for state in vaccines['state'].drop_duplicates():

  vaccines_per_state = vaccines.query(f'state == "{state}"').reset_index(drop=True)
  vaccines_per_state = vaccines_per_state.sort_values(by=['date'])



  vaccines_per_state['primeira_dose_moving_avg_7d'] = np.ceil(vaccines_per_state['primeira_dose'].rolling(window=7).mean())
  vaccines_per_state['primeira_dose_avg_7d_rate_14d'] = vaccines_per_state['primeira_dose_moving_avg_7d']/vaccines_per_state['primeira_dose_moving_avg_7d'].shift(periods=14)

  vaccines_per_state['segunda_dose_moving_avg_7d'] = np.ceil(vaccines_per_state['segunda_dose'].rolling(window=7).mean())
  vaccines_per_state['segunda_dose_avg_7d_rate_14d'] = vaccines_per_state['segunda_dose_moving_avg_7d']/vaccines_per_state['segunda_dose_moving_avg_7d'].shift(periods=14)

  vaccines_per_state['terceira_dose_moving_avg_7d'] = np.ceil(vaccines_per_state['terceira_dose'].rolling(window=7).mean())
  vaccines_per_state['terceira_dose_avg_7d_rate_14d'] = vaccines_per_state['terceira_dose_moving_avg_7d']/vaccines_per_state['terceira_dose_moving_avg_7d'].shift(periods=14)


  if vaccines_is_empty:
    vaccines_ = vaccines_per_state
    vaccines_is_empty = False
  else:
    vaccines_ = pd.concat([vaccines_, vaccines_per_state], ignore_index=True)

vaccines = vaccines_
vaccines_ = None

In [ ]:
vaccines.head(25)

,state,date,primeira_dose,segunda_dose,terceira_dose,population,primeira_dose_acumulado,segunda_dose_acumulado,terceira_dose_acumulado,primeira_dose_acumulado%,segunda_dose_acumulado%,terceira_dose_acumulado%,primeira_dose_moving_avg_7d,primeira_dose_avg_7d_rate_14d,segunda_dose_moving_avg_7d,segunda_dose_avg_7d_rate_14d,terceira_dose_moving_avg_7d,terceira_dose_avg_7d_rate_14d
0,Acre,2021-01-17,5,0,0,881935,5.0,0.0,0.0,0.001,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Acre,2021-01-18,5,0,0,881935,10.0,0.0,0.0,0.001,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Acre,2021-01-19,26,0,0,881935,36.0,0.0,0.0,0.004,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Acre,2021-01-20,947,2,0,881935,983.0,2.0,0.0,0.111,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Acre,2021-01-21,1326,2,0,881935,2309.0,4.0,0.0,0.262,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5,Acre,2021-01-22,951,4,0,881935,3260.0,8.0,0.0,0.37,0.001,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6,Acre,2021-01-23,154,2,0,881935,3414.0,10.0,0.0,0.387,0.001,0.0,488.0,NaN,2.0,NaN,0.0,NaN
7,Acre,2021-01-24,84,2,0,881935,3498.0,12.0,0.0,0.397,0.001,0.0,499.0,NaN,2.0,NaN,0.0,NaN
8,Acre,2021-01-25,752,0,0,881935,4250.0,12.0,0.0,0.482,0.001,0.0,606.0,NaN,2.0,NaN,0.0,NaN
9,Acre,2021-01-26,981,1,0,881935,5231.0,13.0,0.0,0.593,0.001,0.0,743.0,NaN,2.0,NaN,0.0,NaN


In [ ]:
# Troco os nomes das culunas

vaccines.rename(columns={'primeira_dose_acumulado%':'population_1st_dose_%',
                         'segunda_dose_acumulado%':'population_2nd_dose_%',
                         'terceira_dose_acumulado%':'population_3rd_dose_%',
                         'primeira_dose_acumulado': '1st_dose_acumulado',
                         'segunda_dose_acumulado': '2nd_dose_acumulado',
                         'terceira_dose_acumulado': '3rd_dose_acumulado',
                         'primeira_dose': '1st_dose_daily',
                         'segunda_dose': '2nd_dose_daily',
                         'terceira_dose': '3rd_dose_daily'}, inplace=True)

In [ ]:
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

## 5\. Carregamento

In [ ]:
cases.to_csv('./covid-cases.csv', sep=',', index=False)

In [ ]:
vaccines.to_csv('vaccines_21_mp.csv')

## 6.Exploração Interativa de Dados

**KPIs**

1.Casos e mortes nas 24 horas;

2.Média móvel (7 dias) de casos e mortes;

3.Tendência de casos e mortes;

**EDA**

1.Distribuição da média móvel (7 dias) números de primeira doses e mortes ao longo do tempo;

2.Ranking das mortes diárias por estados.

3.Ranking do percentual de vacinação com a priemeira dose, por estados.

## 7 Storytelling

Com os dados disponiblizados, dividimos nosso trabalho em duas etapas: Preparamos e manipulamos os dados atráves do python, e em seguida ultilizamos o looker studio para criarmos o Dashboard. Através do dashboard conseguimos ter uma visualização interativa dos dados, KPI dinâmicos que se altera com o filtro temporal date. KPI:

* Casos(24h): Casos das últimas 24 horas

* Casos(m.m.7d): Casos últilizando a média móvel dos últimos 7 dias

* Mortes(24h): Mortes das últimas 24 horas

* Mortes(m.m.7d): Mortes últilizando a média móvel dos últimos 7 dias

* Casos Tendência: A têndencia dos casos

* Mortes Tendência: Têndencia das mortes.

**Link para o Dashboard** https://lookerstudio.google.com/reporting/d9f08553-57e1-41b5-8048-e9067f05c943